In [1]:
# =======================
# 📦 IMPORTACIONES
# =======================
import warnings
import time
import sys
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import (
    log_loss, accuracy_score, precision_score, recall_score, 
    f1_score, confusion_matrix, roc_auc_score
)

from flwr.client import ClientApp, NumPyClient
from flwr.common import Context, NDArrays, Metrics, Scalar, ndarrays_to_parameters
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.server.strategy import FedAvg
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner

from graphviz import Digraph

from lore_sa.dataset import TabularDataset
from lore_sa.bbox import sklearn_classifier_bbox
from lore_sa.lore import TabularGeneticGeneratorLore
from lore_sa.surrogate.decision_tree import SuperTree

# =======================
# ⚙️ VARIABLES GLOBALES
# =======================
UNIQUE_LABELS = []
FEATURES = []
NUM_SERVER_ROUNDS = 2
NUM_CLIENTS = 2
MIN_AVAILABLE_CLIENTS = 2
fds = None  # Cache del FederatedDataset

# =======================
# 🔧 UTILIDADES MODELO
# =======================

def get_model_parameters(model):
    p = model.get_params()
    return [
        int(p["max_depth"]) if p["max_depth"] is not None else -1,
        int(p["min_samples_split"]),
        int(p["min_samples_leaf"]),
    ]

def set_model_params(model, params):
    max_depth = int(params[0].item()) if hasattr(params[0], "item") else int(params[0])
    min_samples_split = max(2, int(params[1].item()) if hasattr(params[1], "item") else int(params[1]))
    min_samples_leaf = max(1, int(params[2].item()) if hasattr(params[2], "item") else int(params[2]))

    model.set_params(
        max_depth=max_depth if max_depth > 0 else None,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
    )
# =======================
# 🌲 VISUALIZAR SUPERTREE
# =======================

def visualize_supertree(tree, feature_names=None, class_names=None, filename="supertree"):
    dot = Digraph()
    node_id = [0]

    def add_node(node, parent_id=None, edge_label=''):
        curr_id = str(node_id[0])
        node_id[0] += 1

        if node.is_leaf:
            class_index = np.argmax(node.labels)
            class_label = class_names[class_index] if class_names else f"class {class_index}"
            label = f"class: {class_label}\n{node.labels}"
        else:
            fname = f"X_{node.feat}" if feature_names is None else feature_names[node.feat]
            label = f"{fname}"

        dot.node(curr_id, label)

        if parent_id is not None:
            dot.edge(parent_id, curr_id, label=edge_label)

        if not node.is_leaf:
            for i, child in enumerate(node.children):
                label = f"<= {node.intervals[i]:.2f}" if i == 0 else f"> {node.intervals[i - 1]:.2f}"
                add_node(child, curr_id, label)

    add_node(tree)
    dot.render(filename, format='png', cleanup=True)
    print(f"[SERVIDOR] 🌲 SuperTree guardado como '{filename}.png'")

# =======================
# 📄 CONVERTIR ÁRBOL EN TEXTO A NODO
# =======================

def from_text_representation(text: str) -> SuperTree.Node:
    lines = [line.rstrip() for line in text.split("\n") if line.strip()]
    root = None
    stack = []

    for line in lines:
        indent = len(line) - len(line.lstrip())
        level = indent // 4
        content = line.strip()

        if "class:" in content:
            class_info = content.split("class: ")[-1]
            node = SuperTree.Node(is_leaf=True)
            node.predicted_class = class_info
        else:
            feat, cond = content.split(" <= ")
            node = SuperTree.Node(is_leaf=False)
            node.feature = feat.strip()
            node.threshold = float(cond.strip())

        while len(stack) > level:
            stack.pop()

        if stack:
            stack[-1].children.append(node)
        else:
            root = node

        stack.append(node)

    return root

SuperTree.Node.from_text_representation = staticmethod(from_text_representation)

# =======================
# 📥 CARGAR DATOS
# =======================

def load_data(partition_id: int, num_partitions: int):
    global fds, UNIQUE_LABELS, FEATURES

    if fds is None:
        partitioner = IidPartitioner(num_partitions=num_partitions)
        fds = FederatedDataset(dataset="hitorilabs/iris", partitioners={"train": partitioner})

    dataset = fds.load_partition(partition_id, "train").with_format("pandas")[:]
    target_column = dataset.columns[-1]

    if dataset[target_column].dtype == "object":
        label_encoder = LabelEncoder()
        dataset[target_column] = label_encoder.fit_transform(dataset[target_column])
    else:
        dataset[target_column] = dataset[target_column].map({0: "Setosa", 1: "Versicolor", 2: "Virginica"})

    dataset.rename(columns={target_column: "target"}, inplace=True)

    if not UNIQUE_LABELS:
        UNIQUE_LABELS = dataset["target"].unique().tolist()
    if not FEATURES:
        FEATURES = dataset.drop(columns=["target"]).columns.tolist()

    tabular_dataset = TabularDataset(dataset, "target")

    # Train/Test split (80/20)
    X = dataset[FEATURES]
    y = dataset["target"]
    split_idx = int(0.8 * len(X))
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    return X_train, y_train, X_test, y_test, tabular_dataset

# =======================
# 🧪 PRUEBA DE CARGA LOCAL (solo en ejecución directa)
# =======================

if __name__ == "__main__":
    X_train, y_train, X_test, y_test, dataset = load_data(partition_id=0, num_partitions=NUM_CLIENTS)

    print("UNIQUE_LABELS:", UNIQUE_LABELS)
    print("FEATURES:", FEATURES)

    print("\nContenido del TabularDataset:")
    print(dataset.df.head())

    print("\nDescriptor del TabularDataset:")
    print(dataset.descriptor)

2025-04-29 12:34:05,793	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-29 12:34:06,760 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-29 12:34:07,016 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/hitorilabs/iris/resolve/main/README.md HTTP/11" 200 0
2025-04-29 12:34:07,157 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/hitorilabs/iris/resolve/fa62476c42edcf9259f895f43da1a7bf9e2697ae/iris.py HTTP/11" 404 0
2025-04-29 12:34:07,160 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-04-29 12:34:07,597 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/hitorilabs/iris/hitorilabs/iris.py HTTP/11" 404 0
2025-04-29 12:34:07,745 urllib3.connectionpool DEBUG

UNIQUE_LABELS: ['Versicolor', 'Virginica', 'Setosa']
FEATURES: ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']

Contenido del TabularDataset:
   petal_length  petal_width  sepal_length  sepal_width      target
0           4.8          1.8           5.9          3.2  Versicolor
1           3.5          1.0           5.7          2.6  Versicolor
2           5.6          1.4           6.1          2.6   Virginica
3           1.5          0.2           4.6          3.1      Setosa
4           4.9          1.8           6.3          2.7   Virginica

Descriptor del TabularDataset:
{'numeric': {'petal_length': {'index': 0, 'min': 1.100000023841858, 'max': 6.699999809265137, 'mean': 3.9026663, 'std': 1.7214837074279785, 'median': 4.5, 'q1': 1.7000000476837158, 'q3': 5.099999904632568}, 'petal_width': {'index': 1, 'min': 0.10000000149011612, 'max': 2.5, 'mean': 1.228, 'std': 0.7334774732589722, 'median': 1.399999976158142, 'q1': 0.3500000089406967, 'q3': 1.7999999523162842}, 'sep

### Definir el cliente federado con Flower

In [2]:
import warnings
import os
import json
import numpy as np
import pandas as pd
from graphviz import Digraph
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    log_loss, accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from sklearn.exceptions import NotFittedError

from flwr.client import NumPyClient
from flwr.common import Context

from lore_sa.dataset import TabularDataset
from lore_sa.bbox import sklearn_classifier_bbox
from lore_sa.lore import TabularGeneticGeneratorLore
from lore_sa.surrogate.decision_tree import SuperTree


class FlowerClient(NumPyClient):
    def __init__(self, model, X_train, y_train, X_test, y_test, dataset, client_id):
        self.model = model
        self.X_train = X_train.values
        self.y_train = y_train.values
        self.X_test = X_test.values
        self.y_test = y_test.values
        self.dataset = dataset  # Dataset original
        self.unique_labels = np.unique(y_train)
        self.client_id = client_id
        self.received_supertree = None

    def fit(self, parameters, config):
        set_model_params(self.model, parameters)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.model.fit(self.X_train, self.y_train)
        return get_model_parameters(self.model), len(self.X_train), {}

    def evaluate(self, parameters, config):
        set_model_params(self.model, parameters)

        if "supertree" in config:
            try:
                supertree_dict = json.loads(config["supertree"])
                print(f"Cliente {self.client_id} recibiendo SuperTree...")
                self.received_supertree = SuperTree.Node.from_dict(supertree_dict)
            except Exception as e:
                print(f"[CLIENTE {self.client_id}] ❌ Error al recibir SuperTree: {e}")

        try:
            _ = self.model.predict(self.X_test)
        except NotFittedError:
            self.model.fit(self.X_train, self.y_train)

        y_pred = self.model.predict(self.X_test)
        y_proba = self.model.predict_proba(self.X_test)

        # Guardar árbol local
        supertree = SuperTree()
        root_node = supertree.rec_buildTree(self.model, list(range(self.X_train.shape[1])))
        round_number = config.get("server_round", 1)
        self._save_local_tree(root_node, round_number)

        # Guardar árbol en JSON
        tree_json = json.dumps([root_node.to_dict()])
        print(f"[CLIENTE {self.client_id}] ✅ Árbol local generado y enviado")

        # ➡️ Explicabilidad
        self._explain_local_instance()

        return float(log_loss(self.y_test, y_proba)), len(self.X_test), {
            "Accuracy": accuracy_score(self.y_test, y_pred),
            "Precision": precision_score(self.y_test, y_pred, average="weighted", zero_division=1),
            "Recall": recall_score(self.y_test, y_pred, average="weighted"),
            "F1_Score": f1_score(self.y_test, y_pred, average="weighted"),
            "AUC": roc_auc_score(self.y_test, y_proba, multi_class="ovr"),
            "tree_ensemble": tree_json,
        }

    def _explain_local_instance(self):
        # Construir un dataset SOLO con la partición local
        local_df = pd.DataFrame(self.X_train, columns=self.dataset.df.columns[:-1])
        local_df["target"] = self.y_train
        local_tabular_dataset = TabularDataset(local_df, class_name="target")

        bbox = sklearn_classifier_bbox.sklearnBBox(self.model)
        lore = TabularGeneticGeneratorLore(bbox, local_tabular_dataset)

        # Instancia a explicar
        instance = local_tabular_dataset.df.iloc[5][:-1]
        explanation = lore.explain_instance(instance, merge=True)

        rule = explanation["rule"]
        
        print(f"\n📌 Explicabilidad para Cliente {self.client_id}")
        if rule["premises"]:
            regla_str = " AND ".join(
                f"{cond['attr']} {cond['op']} {round(cond['val'], 2)}" for cond in rule["premises"]
            )
            regla_str += f" → {rule['consequence']['val']}"
        else:
            regla_str = f"Siempre → {rule['consequence']['val']}"

        print(f"🔍 Regla: {regla_str}")

    def _save_local_tree(self, tree, round_number):
        self._save_tree(
            tree,
            round_number,
            f"arbol_local_cliente_{self.client_id}_ronda_{round_number}",
            f"ArbolLocal_Cliente_{self.client_id}"
        )

    def _save_tree(self, root_node, round_number, filename, subfolder):
        dot = Digraph()
        node_id = [0]

        def add_node(node, parent_id=None, edge_label=""):
            curr_id = str(node_id[0])
            node_id[0] += 1

            if node.is_leaf:
                class_index = np.argmax(node.labels)
                class_label = str(self.unique_labels[class_index]) if len(self.unique_labels) > 0 else f"class {class_index}"
                label = f"class: {class_label}\n{node.labels}"
            else:
                fname = self.dataset.df.columns[node.feat] if node.feat is not None else "?"
                label = f"{fname}"

            dot.node(curr_id, label)

            if parent_id:
                dot.edge(parent_id, curr_id, label=edge_label)

            if hasattr(node, "children") and hasattr(node, "intervals"):
                for i, child in enumerate(node.children):
                    thr_label = f"<= {node.intervals[i]:.2f}" if i == 0 else f"> {node.intervals[i-1]:.2f}"
                    add_node(child, curr_id, thr_label)
            elif hasattr(node, "_left_child") or hasattr(node, "_right_child"):
                if node._left_child:
                    add_node(node._left_child, curr_id, f"<= {node.thresh:.2f}")
                if node._right_child:
                    add_node(node._right_child, curr_id, f"> {node.thresh:.2f}")

        add_node(root_node)

        round_folder = f"Ronda_{round_number}"
        os.makedirs(round_folder, exist_ok=True)

        folder_path = f"{round_folder}/{subfolder}"
        os.makedirs(folder_path, exist_ok=True)

        filepath = f"{folder_path}/{filename}"
        dot.render(filepath, format="png", cleanup=True)
        print(f"[CLIENTE {self.client_id}] ✅ Árbol guardado como '{filepath}.png'")


def create_model():
    return DecisionTreeClassifier(max_depth=5, min_samples_split=2, random_state=42)


def client_fn(context: Context):
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]
    X_train, y_train, X_test, y_test, dataset = load_data(partition_id, num_partitions)
    model = create_model()
    return FlowerClient(model, X_train, y_train, X_test, y_test, dataset, client_id=partition_id + 1).to_client()


client_app = ClientApp(client_fn=client_fn)


# Configurar el Servidor de Flower

In [3]:
# ============================
# 📦 IMPORTACIONES NECESARIAS
# ============================
import os
import time
import json
import numpy as np
from typing import List, Tuple, Dict
from sklearn.tree import DecisionTreeClassifier

from flwr.common import Context, Metrics, Scalar, ndarrays_to_parameters
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.server.strategy import FedAvg

from graphviz import Digraph
from lore_sa.surrogate.decision_tree import SuperTree

# ============================
# ⚖️ CONFIGURACIÓN GLOBAL
# ============================
MIN_AVAILABLE_CLIENTS = 2
NUM_SERVER_ROUNDS = 2
FEATURES = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
UNIQUE_LABELS = ["Setosa", "Versicolor", "Virginica"]
LATEST_SUPERTREE_JSON = None  # 🌲 Guardar árbol generado

# ============================
# 🧐 MODELO Y UTILIDADES
# ============================

def create_model():
    return DecisionTreeClassifier(max_depth=2, random_state=42)

def get_model_parameters(model):
    p = model.get_params()
    return [p["max_depth"] or -1, p["min_samples_split"], p["min_samples_leaf"]]

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Dict[str, Scalar]:
    total = sum(n for n, _ in metrics)
    avg: Dict[str, List[float]] = {}
    for n, met in metrics:
        for k, v in met.items():
            if isinstance(v, (float, int)):
                avg.setdefault(k, []).append(n * float(v))
    return {k: sum(vs) / total for k, vs in avg.items()}

# ============================
# 🚀 SERVIDOR FLOWER
# ============================

def server_fn(context: Context) -> ServerAppComponents:
    model = create_model()
    initial_params = ndarrays_to_parameters(get_model_parameters(model))

    strategy = FedAvg(
        min_available_clients=MIN_AVAILABLE_CLIENTS,
        fit_metrics_aggregation_fn=weighted_average,
        evaluate_metrics_aggregation_fn=weighted_average,
        initial_parameters=initial_params,
    )

    strategy.configure_fit = _inject_round(strategy.configure_fit)
    strategy.configure_evaluate = _inject_round(strategy.configure_evaluate)

    original_aggregate = strategy.aggregate_evaluate

    def custom_aggregate_evaluate(server_round, results, failures):
        global LATEST_SUPERTREE_JSON
        aggregated_metrics = original_aggregate(server_round, results, failures)

        try:
            print(f"\n[SERVIDOR] 🌲 Generando SuperTree - Ronda {server_round}")
            tree_dicts = []
            total_arboles = 0

            for client_idx, (_, evaluate_res) in enumerate(results):
                metrics = evaluate_res.metrics
                trees_json = metrics.get("tree_ensemble", None)
                if trees_json:
                    try:
                        trees_list = json.loads(trees_json)
                        for tdict in trees_list:
                            root = SuperTree.Node.from_dict(tdict)
                            if root:
                                tree_dicts.append(root)
                                total_arboles += 1
                    except Exception as e:
                        print(f"[CLIENTE {client_idx+1}] ❌ Error al parsear árbol: {e}")

            print(f"[SERVIDOR] 📊 Total de árboles: {total_arboles}")

            if not tree_dicts:
                print("[SERVIDOR] ⚠️ No se recibieron árboles. Se omite SuperTree.")
                return aggregated_metrics

            supertree = SuperTree()
            supertree.mergeDecisionTrees(tree_dicts, num_classes=len(UNIQUE_LABELS), feature_names=FEATURES)
            supertree.prune_redundant_leaves_full()
            supertree.merge_equal_class_leaves()

            _save_supertree_plot(supertree.root, server_round, feature_names=FEATURES, class_names=UNIQUE_LABELS)
            LATEST_SUPERTREE_JSON = json.dumps(supertree.root.to_dict())

        except Exception as e:
            print(f"[SERVIDOR] ❌ Error en SuperTree: {e}")

        time.sleep(10)
        return aggregated_metrics

    strategy.aggregate_evaluate = custom_aggregate_evaluate
    config = ServerConfig(num_rounds=NUM_SERVER_ROUNDS)
    return ServerAppComponents(strategy=strategy, config=config)

# ============================
# 📂 HELPERS
# ============================

def _inject_round(original_fn):
    def wrapper(server_round, parameters, client_manager):
        global LATEST_SUPERTREE_JSON
        instructions = original_fn(server_round, parameters, client_manager)
        for _, ins in instructions:
            ins.config["server_round"] = server_round
            if LATEST_SUPERTREE_JSON:
                ins.config["supertree"] = LATEST_SUPERTREE_JSON
        return instructions
    return wrapper

def _save_supertree_plot(root_node, round_number, feature_names=None, class_names=None):
    round_folder = f"Ronda_{round_number}"
    os.makedirs(round_folder, exist_ok=True)

    supertree_folder = f"{round_folder}/Supertree"
    os.makedirs(supertree_folder, exist_ok=True)

    dot = Digraph()
    node_id = [0]

    def add_node(node, parent=None, label=""):
        curr = str(node_id[0])
        node_id[0] += 1

        if node.is_leaf:
            class_index = np.argmax(node.labels)
            class_label = class_names[class_index] if class_names else f"Clase {class_index}"
            label_text = f"Clase: {class_label}\n{node.labels}"
        else:
            fname = f"X_{node.feat}" if feature_names is None else feature_names[node.feat]
            label_text = f"{fname}"

        dot.node(curr, label_text)

        if parent:
            dot.edge(parent, curr, label=label)

        if not node.is_leaf:
            for i, child in enumerate(node.children):
                thr_label = f"<= {node.intervals[i]:.2f}" if i == 0 else f"> {node.intervals[i - 1]:.2f}"
                add_node(child, curr, thr_label)

    add_node(root_node)
    filename = f"{supertree_folder}/supertree_ronda_{round_number}"
    dot.render(filename, format="png", cleanup=True)
    print(f"[SERVIDOR] ✅ SuperTree guardado como '{filename}.png'")

# ============================
# 🔧 INICIALIZAR SERVER APP
# ============================
server_app = ServerApp(server_fn=server_fn)


**Pasos que se realizan en el notebook:**

1. El servidor inicializa el modelo y lo envía a cada uno de los clientes.

2. Cada cliente entrena un RandomForest con su respectivo subconjunto de datos o partición que hemos realizado al principio.

3. Los clientes entrenan, y mandan sus hiperparámetros (Nº de árboles, profundidad, etc.) al servidor.

4. El servidor combina los parámetros y actualiza el modelo global.

5. Se mide el rendimiento del modelo sobre cada cliente, obteniendo también sus contrafactuales y se repite el proceso las rondas que deseemos.

# Ejecutar la Simulación Federada


In [4]:
from flwr.simulation import run_simulation
import logging
import warnings
import ray

warnings.filterwarnings("ignore", category=DeprecationWarning)


logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger("filelock").setLevel(logging.WARNING)
logging.getLogger("ray").setLevel(logging.WARNING)
logging.getLogger('graphviz').setLevel(logging.WARNING)
# logging.getLogger("flwr").setLevel(logging.WARNING)




ray.shutdown()  # Apagar cualquier sesión previa de Ray
ray.init(local_mode=True)  # Desactiva multiprocessing, usa un solo proceso principal

backend_config = {"num_cpus": 1}

run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)


2025-04-29 12:34:14,446	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2025-04-29 12:34:18,117 flwr         DEBUG    Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
:job_id:01000000
INFO :      [INIT]
:actor_name:ClientAppActor
:actor_name:ClientAppActor
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
:actor_name:ClientAppActor
:actor_name:ClientAppActor
INFO :      Evaluation returned no results (`None`)
:actor_name:ClientAppActor
INFO :      
:actor_name:ClientAppActor
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


:job_id:01000000
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


[CLIENTE 1] ✅ Árbol guardado como 'Ronda_1/ArbolLocal_Cliente_1/arbol_local_cliente_1_ronda_1.png'
[CLIENTE 1] ✅ Árbol local generado y enviado
[CLIENTE 2] ✅ Árbol guardado como 'Ronda_1/ArbolLocal_Cliente_2/arbol_local_cliente_2_ronda_1.png'
[CLIENTE 2] ✅ Árbol local generado y enviado


INFO :      aggregate_evaluate: received 2 results and 0 failures


...

INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


Cliente 1 recibiendo SuperTree...
Cliente 2 recibiendo SuperTree...
[CLIENTE 1] ✅ Árbol guardado como 'Ronda_2/ArbolLocal_Cliente_1/arbol_local_cliente_1_ronda_2.png'
[CLIENTE 1] ✅ Árbol local generado y enviado
[CLIENTE 2] ✅ Árbol guardado como 'Ronda_2/ArbolLocal_Cliente_2/arbol_local_cliente_2_ronda_2.png'
[CLIENTE 2] ✅ Árbol local generado y enviado

📌 Explicabilidad para Cliente 2
🔍 Regla: sepal_length <= 2.52 → Setosa


INFO :      aggregate_evaluate: received 2 results and 0 failures



📌 Explicabilidad para Cliente 1
🔍 Regla: sepal_length > 4.94 → Virginica

[SERVIDOR] 🌲 Generando SuperTree - Ronda 2
[SERVIDOR] 📊 Total de árboles: 2
[SERVIDOR] ✅ SuperTree guardado como 'Ronda_2/Supertree/supertree_ronda_2.png'


INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 132.69s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.220446049250313e-16
INFO :      		round 2: 2.220446049250313e-16
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'AUC': [(1, 1.0), (2, 1.0)],
INFO :      	 'Accuracy': [(1, 1.0), (2, 1.0)],
INFO :      	 'F1_Score': [(1, 1.0), (2, 1.0)],
INFO :      	 'Precision': [(1, 1.0), (2, 1.0)],
INFO :      	 'Recall': [(1, 1.0), (2, 1.0)]}
INFO :      
